In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


import time
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS
import os 
import PyPDF2

from langchain.vectorstores import Chroma
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.vectorstores import Qdrant
from langchain_community.vectorstores import Chroma

In [2]:
load_dotenv()

# # # Set the environment variables for Azure OpenAI
os.environ["OPENAI_API_TYPE"] = os.getenv("")
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = os.getenv("")
os.environ["OPENAI_API_VERSION"] = os.getenv("")


In [ ]:
# Create an instance of AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv(""),
    model="",
    openai_api_base= "",
    openai_api_key= os.getenv(""),
    openai_api_version = os.getenv(""),
    openai_api_type=os.getenv("")
)

In [ ]:
model = AzureChatOpenAI(
        openai_api_base=os.getenv("GPT_4_API_BASE"),
        openai_api_version=os.getenv("GPT_4_API_VERSION"),
        deployment_name=os.getenv("GPT_4_ID"),
        openai_api_key=os.getenv("GPT_4_API_KEY"),
        openai_api_type=os.getenv("ORGANIZATION"),
    )

In [5]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR
import json

pdf = PDF(src="")
ocr = TesseractOCR(lang="eng")
pdf_tables = pdf.extract_tables(ocr=ocr)
pdf.to_xlsx("rel.xlsx",ocr=ocr)

import pandas as pd
import json

def xlsx_to_json(file_path):
    # Read the Excel file
    xls = pd.ExcelFile(file_path)

    # Initialize dictionary to hold all sheet data
    all_data = {}

    # Iterate through each sheet in the Excel file
    for sheet_name in xls.sheet_names:
        # Read each sheet into a DataFrame
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Convert the DataFrame rows to list of dictionaries
        sheet_data = df.to_dict(orient='records')

        # Add the data to the dictionary using sheet name as the key
        all_data[sheet_name] = sheet_data

    return all_data

# Replace 'file_path.xlsx' with the path to your Excel file
file_path = 'rel.xlsx'

# Convert Excel data to JSON
json_data = xlsx_to_json(file_path)

# Print or do whatever you want with the JSON data
mapping_tabular_data = json.loads(json.dumps(json_data, indent=4))


tesseract 5.3.3-30-gea0b
 leptonica-1.79.0
  libgif 5.1.4 : libjpeg 8d (libjpeg-turbo 2.0.3) : libpng 1.6.37 : libtiff 4.1.0 : zlib 1.2.11 : libwebp 0.6.1 : libopenjp2 2.3.1
 Found AVX512BW
 Found AVX512F
 Found AVX512VNNI
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.4.0 zlib/1.2.11 liblzma/5.2.4 bz2lib/1.0.8 liblz4/1.9.2 libzstd/1.4.4
 Found libcurl/7.68.0 OpenSSL/1.1.1f zlib/1.2.11 brotli/1.0.7 libidn2/2.2.0 libpsl/0.21.0 (+libidn2/2.2.0) libssh/0.9.3/openssl/zlib nghttp2/1.40.0 librtmp/2.3


In [6]:
mapping_tabular_data = {i.replace(" ",""):v for i,v in mapping_tabular_data.items()}
# mapping_tabular_data.keys()

In [7]:
import re

def clean_text(input_text):
    # Replace '\n' with space
    cleaned_text = input_text.replace("\n", " ")
    cleaned_text = cleaned_text.replace("\\", " ")
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    return cleaned_text


In [8]:
import fitz
def extractTextFromPdf():
    files =  ['']
    for count,file in enumerate(files):
        try:
            if file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""#[]
                for page in range(len(pdf_reader.pages)):
                    # text += pdf_reader.pages[page].extract_text()
                    new_page = clean_text(pdf_reader.pages[page].extract_text())
    
                    if count==0:
                        for key_name in mapping_tabular_data.keys():
                            if "Page"+str(page+1)+"-" in key_name:
                                new_page = new_page+ clean_text(str({'TABLE_DATA':mapping_tabular_data[key_name]}))
                                print("Page"+str(page+1)+"-",key_name)
                    # print(new_page)   
                    # text.append(new_page)
                    text = text+new_page
                
                if text and not text == '':
                    # text.append(json.dumps(mapping_tabular_data))
                    return text
                else:
                    return None
            else:
                return None
    
        except Exception as e:
            print(f"An error occurred: {e}")

    return None

In [10]:
extracted_text = extractTextFromPdf()
from langchain_core.documents.base import Document
documents = []
# for c,i in enumerate(a):
#     new_doc = Document(page_content=i, metadata={"page_number":c+1})
#     documents.append(new_doc)

new_doc = Document(page_content=extracted_text, metadata={"page_number":1})
documents.append(new_doc)


Page4- Page4-Table1
Page4- Page4-Table2
Page4- Page4-Table3
Page5- Page5-Table1
Page5- Page5-Table2
Page6- Page6-Table1
Page6- Page6-Table2
Page6- Page6-Table3
Page7- Page7-Table1
Page8- Page8-Table1


In [11]:
db = Chroma.from_documents(documents, embeddings)

In [12]:
chroma_retriever = db.as_retriever(search_kwargs={'k': 1})

In [13]:
query = "create mapping for reverse file format"
docs = db.similarity_search(query)

# print results
# print(docs)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [ ]:
question = ""
mapping_extraction_query = f"""
Question : {question}

{{context}}


        
"""
prompt = ChatPromptTemplate.from_template(mapping_extraction_query)    
chain = (
    {"context": chroma_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

response = chain.invoke(question)
print(response)